# Clustering Samples

Script to cluster and label all the samples of all the studies (given by their geo id)

@authors: nLp ATTACK Luis, Arun, Claire, and Karsten

April 01 2019--April 05 2019

## Import modules and dependencies

In [2]:
import pandas as pd
import numpy as np
import sklearn.cluster
import distance # first, >>> pip install Distance
import time
import itertools

## Define functions

### clustering function
aim: to cluster the sample titles into clusters of samples with similar names

In [3]:
def cluster_terms(input_words):
    """
    to cluster a list of word strings (uses levenshtein distance with affinity propagation)
    
    returns a list of the clusters. each list is a dictionary. each dictionary's key is the exemplar word,
    and value is a list of the words in the cluster
       
    (list --> list[dictionary{string: list}])
    
    example
    >>> text = ['apple', 'oranges', 'ornnges', 'melons', 'aaple', 'apples', 
                 'melon', 'apale', 'meeons', 'orange', 'orage', 'meeon', 'melan']
    >>> print(cluster_terms(text))
    [{'apple': ['aaple', 'apale', 'apple', 'apples']}, 
    {'melon': ['meeon', 'meeons', 'melan', 'melon', 'melons']}, 
    {'orange': ['orage', 'orange', 'oranges', 'ornnges']}]
    """
    words = np.asarray(input_words) #So that indexing with a list will work
    lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

    affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping=0.5)
    affprop.fit(lev_similarity)
    
    clustered_terms = []    
    for cluster_id in np.unique(affprop.labels_):
        cluster_dict = {}
        exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
        nth_cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
        cluster_dict[exemplar] = list(nth_cluster)
        clustered_terms.append(cluster_dict)
    return clustered_terms

### labeling functions
aim: to label the clustered sample titles into the correct names

note: the implementation labels as per the *most common phrase* in the sample titles. does not work perfectly, needs minor fixes

In [4]:
def is_substr(find, data):
    if len(data) < 1 and len(find) < 1:        
        return False
    for i in range(len(data)):
        if find not in data[i]:            
            return False
    return True

def long_substr(data):
    """
    returns the longest common substring from a list of strings
    """
    substr = ''
    if len(data) > 1 and len(data[0]) > 0:
        for i in range(len(data[0])):
            for j in range(len(data[0])-i+1):
                if j > len(substr) and is_substr(data[0][i:i+j], data):                    
                    substr = data[0][i:i+j]
    return substr

## Input given data

In [5]:
# Read samples data
samples_df = pd.read_pickle('../data/interim/samples.pkl')

## Run data
### Example 1 (samples <= 10)

In [6]:
geo_id_samples = samples_df.loc[samples_df['geo_id'] == '200003505']
print(geo_id_samples, '\n')
sample_titles = list(geo_id_samples.title)
start_time = time.time() # to monitor time taken
clusters = (cluster_terms(sample_titles))
print('\nNo. of clusters: ', len(clusters), '\n')
for item in clusters:
    print(item, '\n')
    clustered_sample_titles = list(item.values())[0]    
    print('label: ', long_substr(clustered_sample_titles), '\n')
end_time = time.time() # to monitor time taken
print('time taken: ', end_time - start_time, 'seconds')

          geo_id  nsamples        date accession  \
36227  200003505         9  2005/10/27  GSM79992   
36228  200003505         9  2005/10/27  GSM80027   
36229  200003505         9  2005/10/27  GSM80024   
36230  200003505         9  2005/10/27  GSM79993   
36231  200003505         9  2005/10/27  GSM80025   
36232  200003505         9  2005/10/27  GSM79783   
36233  200003505         9  2005/10/27  GSM80026   
36234  200003505         9  2005/10/27  GSM79782   
36235  200003505         9  2005/10/27  GSM80023   

                                                 title  
36227      23353, GATA2 overexpresed, FLT3 non-mutated  
36228          20531, GATA2 overexpresed, FLT3 mutated  
36229  26611, GATA2 non-overexpresed, FLT3 non-mutated  
36230  16320, GATA2 non-overexpresed, FLT3 non-mutated  
36231          16583, GATA2 overexpresed, FLT3 mutated  
36232      20505, GATA2 overexpresed, FLT3 non-mutated  
36233          18788, GATA2 overexpresed, FLT3 mutated  
36234      14413, GATA2

### Example 2 (10 <= samples <= 100)

In [7]:
geo_id_samples = samples_df.loc[samples_df['geo_id'] == '200003484']
print(geo_id_samples)
sample_titles = list(geo_id_samples.title)
start_time = time.time() # to monitor time taken
clusters = (cluster_terms(sample_titles))
print('\nNo. of clusters: ', len(clusters), '\n')
for item in clusters:
    print(item, '\n')
    clustered_sample_titles = list(item.values())[0]    
    print('label: ', long_substr(clustered_sample_titles), '\n')
end_time = time.time() # to monitor time taken
print('time taken: ', end_time - start_time, 'seconds')

          geo_id  nsamples        date accession                   title
35191  200003484        30  2006/01/20  GSM78851           LSPM2 Control
35192  200003484        30  2006/01/20  GSM78864           MM455 Control
35193  200003484        30  2006/01/20  GSM78839         D23 24 h PEP005
35194  200003484        30  2006/01/20  GSM78842         D24 24 h PEP005
35195  200003484        30  2006/01/20  GSM78859          MM253 24 h TPA
35196  200003484        30  2006/01/20  GSM78848   LSPM2 24 h TPA + 24 h
35197  200003484        30  2006/01/20  GSM78857           MM127 Control
35198  200003484        30  2006/01/20  GSM78854   MM127 24 h TPA + 24 h
35199  200003484        30  2006/01/20  GSM78837            D04 24 h TPA
35200  200003484        30  2006/01/20  GSM78840            D23 24 h TPA
35201  200003484        30  2006/01/20  GSM78860           MM253 Control
35202  200003484        30  2006/01/20  GSM78843            D24 24 h TPA
35203  200003484        30  2006/01/20  GSM78846   

### Example 3 (100 <= samples <= 500)

In [8]:
geo_id_samples = samples_df.loc[samples_df['geo_id'] == '200128147']
print(geo_id_samples)
sample_titles = list(geo_id_samples.title)
start_time = time.time() # to monitor time taken
clusters = (cluster_terms(sample_titles))
print('\nNo. of clusters: ', len(clusters), '\n')
for item in clusters:
    print(item, '\n')
    clustered_sample_titles = list(item.values())[0]    
    print('label: ', long_substr(clustered_sample_titles), '\n')
end_time = time.time() # to monitor time taken
print('time taken: ', end_time - start_time, 'seconds') # time taken is typically 32.016 seconds

            geo_id  nsamples        date   accession  \
2048837  200128147       192  2019/03/12  GSM3665739   
2048838  200128147       192  2019/03/12  GSM3665716   
2048839  200128147       192  2019/03/12  GSM3665618   
2048840  200128147       192  2019/03/12  GSM3665741   
2048841  200128147       192  2019/03/12  GSM3665724   
2048842  200128147       192  2019/03/12  GSM3665641   
2048843  200128147       192  2019/03/12  GSM3665610   
2048844  200128147       192  2019/03/12  GSM3665747   
2048845  200128147       192  2019/03/12  GSM3665658   
2048846  200128147       192  2019/03/12  GSM3665633   
2048847  200128147       192  2019/03/12  GSM3665619   
2048848  200128147       192  2019/03/12  GSM3665620   
2048849  200128147       192  2019/03/12  GSM3665698   
2048850  200128147       192  2019/03/12  GSM3665701   
2048851  200128147       192  2019/03/12  GSM3665626   
2048852  200128147       192  2019/03/12  GSM3665686   
2048853  200128147       192  2019/03/12  GSM366

### Example 4 (500 <= samples <= 1000)

In [9]:
geo_id_samples = samples_df.loc[samples_df['geo_id'] == '200000014']
print(geo_id_samples)
sample_titles = list(geo_id_samples.title)
start_time = time.time() # to monitor time taken
clusters = (cluster_terms(sample_titles))
print('\nNo. of clusters: ', len(clusters), '\n')
for item in clusters:
    print(item, '\n')
    clustered_sample_titles = list(item.values())[0]    
    print('label: ', long_substr(clustered_sample_titles), '\n')
end_time = time.time() # to monitor time taken
print('time taken: ', end_time - start_time, 'seconds') # time taken is typically 414.407 seconds

         geo_id  nsamples        date  accession  \
384   200000014       765  2001/11/29  GSM383758   
385   200000014       765  2001/11/29  GSM383993   
386   200000014       765  2001/11/29  GSM384279   
387   200000014       765  2001/11/29  GSM384196   
388   200000014       765  2001/11/29  GSM383910   
389   200000014       765  2001/11/29  GSM383827   
390   200000014       765  2001/11/29  GSM383721   
391   200000014       765  2001/11/29  GSM384007   
392   200000014       765  2001/11/29  GSM384133   
393   200000014       765  2001/11/29  GSM384362   
394   200000014       765  2001/11/29  GSM384076   
395   200000014       765  2001/11/29  GSM383970   
396   200000014       765  2001/11/29  GSM383847   
397   200000014       765  2001/11/29  GSM383741   
398   200000014       765  2001/11/29  GSM384325   
399   200000014       765  2001/11/29  GSM384219   
400   200000014       765  2001/11/29  GSM384113   
401   200000014       765  2001/11/29  GSM384382   
402   200000

### Example 5 (only 1 cluster)

In [10]:
geo_id_samples = samples_df.loc[samples_df['geo_id'] == '200002487']
print(geo_id_samples)
sample_titles = list(geo_id_samples.title)
start_time = time.time() # to monitor time taken
clusters = (cluster_terms(sample_titles))
print('\nNo. of clusters: ', len(clusters), '\n')
for item in clusters:
    print(item, '\n')
    clustered_sample_titles = list(item.values())[0]    
    print('label: ', long_substr(clustered_sample_titles), '\n')
end_time = time.time() # to monitor time taken
print('time taken: ', end_time - start_time, 'seconds')

          geo_id  nsamples        date accession        title
24316  200002487        10  2005/05/15  GSM42172      ITMST_0
24317  200002487        10  2005/05/15  GSM42178  ITME6E7ST_0
24318  200002487        10  2005/05/15  GSM42104        ITM_0
24319  200002487        10  2005/05/15  GSM42170        ITM_1
24320  200002487        10  2005/05/15  GSM42176    ITME6E7_0
24321  200002487        10  2005/05/15  GSM42179  ITME6E7ST_1
24322  200002487        10  2005/05/15  GSM42103        ITV_1
24323  200002487        10  2005/05/15  GSM42177    ITME6E7_1
24324  200002487        10  2005/05/15  GSM42175      ITMST_1
24325  200002487        10  2005/05/15  GSM42080        ITV_0

No. of clusters:  1 

{'ITMST_0': ['ITME6E7ST_0', 'ITME6E7ST_1', 'ITME6E7_0', 'ITME6E7_1', 'ITMST_0', 'ITMST_1', 'ITM_0', 'ITM_1', 'ITV_0', 'ITV_1']} 

label:  IT 

time taken:  0.018591880798339844 seconds


### Example 6 (inaccurately clustered)

In [33]:
geo_id_samples = samples_df.loc[samples_df['geo_id'] == '200001572']
print(geo_id_samples)
sample_titles = list(geo_id_samples.title)
start_time = time.time() # to monitor time taken
clusters = (cluster_terms(sample_titles))
print('\nNo. of clusters: ', len(clusters), '\n')
for item in clusters:
    print(item, '\n')
    clustered_sample_titles = list(item.values())[0]    
    print('label: ', long_substr(clustered_sample_titles), '\n')
end_time = time.time() # to monitor time taken
print('time taken: ', end_time - start_time, 'seconds')

          geo_id  nsamples        date accession                     title
13590  200001572        30  2004/07/21  GSM27021          37 year old male
13591  200001572        30  2004/07/21  GSM27024          42 year old male
13592  200001572        30  2004/07/21  GSM27044       106 year old female
13593  200001572        30  2004/07/21  GSM27038          85 year old male
13594  200001572        30  2004/07/21  GSM27023          40 year old male
13595  200001572        30  2004/07/21  GSM27015  another 26 year old male
13596  200001572        30  2004/07/21  GSM27026        48 year old female
13597  200001572        30  2004/07/21  GSM27029        56 year old female
13598  200001572        30  2004/07/21  GSM27043          95 year old male
13599  200001572        30  2004/07/21  GSM27040          90 year old male
13600  200001572        30  2004/07/21  GSM27041        90 year old female
13601  200001572        30  2004/07/21  GSM27017        27 year old female
13602  200001572        3

### Example 7 (clustering failure)

In [32]:
geo_id_samples = samples_df.loc[samples_df['geo_id'] == '200001428']
print(geo_id_samples)
sample_titles = list(geo_id_samples.title)
start_time = time.time() # to monitor time taken
clusters = (cluster_terms(sample_titles))
print('\nNo. of clusters: ', len(clusters), '\n')
for item in clusters:
    print(item, '\n')
    clustered_sample_titles = list(item.values())[0]    
    print('label: ', long_substr(clustered_sample_titles), '\n')
end_time = time.time() # to monitor time taken
print('time taken: ', end_time - start_time, 'seconds')

          geo_id  nsamples        date accession title
11648  200001428        22  2004/08/15  GSM23742  413J
11649  200001428        22  2004/08/15  GSM23751  413S
11650  200001428        22  2004/08/15  GSM23748  413P
11651  200001428        22  2004/08/15  GSM23745  413M
11652  200001428        22  2004/08/15  GSM23753  413U
11653  200001428        22  2004/08/15  GSM23747  413O
11654  200001428        22  2004/08/15  GSM23744  413L
11655  200001428        22  2004/08/15  GSM23750  413R
11656  200001428        22  2004/08/15  GSM23756  413X
11657  200001428        22  2004/08/15  GSM23736  413C
11658  200001428        22  2004/08/15  GSM23741  413I
11659  200001428        22  2004/08/15  GSM23738  413F
11660  200001428        22  2004/08/15  GSM23755  413W
11661  200001428        22  2004/08/15  GSM23752  413T
11662  200001428        22  2004/08/15  GSM23735  413B
11663  200001428        22  2004/08/15  GSM23746  413N
11664  200001428        22  2004/08/15  GSM23743  413K
11665  200

/home/arun/.anaconda3/lib/python3.7/site-packages/sklearn/cluster/affinity_propagation_.py:125: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn("All samples have mutually equal similarities. "


## Problems

The following problems were noted but not resolved because of lack of time.

**Clustering**

(1) Clustering is not always accurate.

(2) Clustering can be slow. For example, it took 5 minutes to cluster 750 samples.

**Labeling**

(1) **{Minor}** Labeling is not accurate and small changes need to be made. For example, the current implementation takes the _longest common substring_. But this means that an example list such as ['Ovary 0314', 'Ovary 0408', 'Ovary 0466', 'Pancreas 0432'] gives the label "a" instead of "Ovary"! Thus, the _most common substring_ (or something else) should be considered rather than the _longest common substring_.
    
(2) **{Major}** Even when labeling is accurate, the desired labeling of _control vs. non-control_ or _healthy vs. diseased_ is not achieved.

**Others**

(1) Validation code (or some kind of test dataset) is needed to verify if the clusters are acceptable.

## Improvements

The following improvements are suggested:

**Clustering**

(1) Do some data preprocessing.

(2) Use more advanced clustering methods and/or other edit distances.

(3) Use a O(log(n)) method like simHash.

**Labeling**

(1) Use _most common substring_ or something else to label the samples.
    
(2) Use a heuristic to label the samples _after_ they are clustered. For example, (match titles with a predefined vocabulary such as ['control', 'not infected', 'WT']).

(3) Q: Can the exemplar be the label?

**Others**

(1) Prepare manually tagged test dataset for verification.